In [1]:
import os
from os import listdir
from PIL import Image
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from keras.models import load_model
import numpy as np

import pickle
import cv2

In [2]:
#HaarCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = load_model('facenet_keras.h5')

In [12]:
def extract_face(filename):
    image = cv2.imread(filename)
    
    #imagegray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    wajah = face_cascade.detectMultiScale(image,1.1,4)
    
    if len(wajah)>0:
        x1, y1, width, height = wajah[0]         
    else:
        x1, y1, width, height = 1, 1, 10, 10
        
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)                  # konversi dari OpenCV ke PIL
    image = asarray(image)
    face = image[y1:y2, x1:x2]                         # extract the face
    
    image = Image.fromarray(face)                       # resize pixels to the model size
    image = image.resize((160,160))
    face_array = asarray(image)
    return face_array

In [13]:
folder='train/'
database = {}

for filename in listdir(folder):

    path = folder + filename
    face = extract_face(path)
    
    face_pixels = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    
    face = expand_dims(face, axis=0)
    signature = model.predict(face)
    
    database[os.path.splitext(filename)[0]]=signature


In [14]:
myfile = open("data.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()